In [1]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import math
import copy

from google.colab import drive
drive.mount('/content/drive')

class MLP(nn.Module):
  def __init__(self, n_units, init_scale=1.0):
    super(MLP, self).__init__()

    self._n_units = copy.copy(n_units)
    self._layers = []
    for i in range(1, len(n_units)):
      layer = nn.Linear(n_units[i-1], n_units[i], bias=False)
      variance = math.sqrt(2.0 / (n_units[i-1] + n_units[i]))
      layer.weight.data.normal_(0.0, init_scale * variance)
      self._layers.append(layer)

      name = 'fc%d' % i
      if i == len(n_units) - 1:
        name = 'fc'  # the prediction layer is just called fc
      self.add_module(name, layer)


  def forward(self, x):
    x = x.view(-1, self._n_units[0])
    out = self._layers[0](x)
    for layer in self._layers[1:]:
      out = F.relu(out)
      out = layer(out)
    return out



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive/'My Drive'

/content/drive/My Drive


In [0]:
def get_data():
  normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]], std=[x / 255.0 for x in [63.0, 62.1, 66.7]])
  transform_train = transforms.Compose([
          transforms.ToTensor(),
          normalize])
  transform_test = transforms.Compose([
        transforms.ToTensor(),
        normalize])
  train_dataset = datasets.CIFAR10(root='data', 
                                train=True, 
                                transform=transform_train,
                                download=True)
  test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transform_test,
                                download=True)  
  train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=128,
                          num_workers=4,
                          shuffle=True)
  test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=128,
                         num_workers=4,
                         shuffle=False)
  return train_loader, test_loader

In [0]:
def compute_acc(model, data_loader):
    correct_pred, num_examples = 0, 0
    model.eval()
    for i, (features, targets) in enumerate(data_loader):
        probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        assert predicted_labels.size() == targets.size()
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [5]:
weights = []
models = []

seeds = 10

for i in range(seeds):
  model = torch.load('MLP_truelabels_seed' + str(i), map_location=lambda storage, loc: storage)
  #train_loader, test_loader = get_data()
  
  models.append(model)
  weights.append(model.fc1.weight)

  #train_acc = compute_acc(model, train_loader)        
  #print(f'train ACC: {train_acc:.2f}, seed: {i}')
        
  #test_acc = compute_acc(model, test_loader)        
  #print(f'Test ACC: {test_acc:.2f}')

Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.99, seed: 0
Test ACC: 53.14
Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.98, seed: 1
Test ACC: 53.21
Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.99, seed: 2
Test ACC: 53.60
Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.98, seed: 3
Test ACC: 53.35
Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.98, seed: 4
Test ACC: 52.81
Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.98, seed: 5
Test ACC: 52.66
Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.98, seed: 6
Test ACC: 52.86
Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.98, seed: 7
Test ACC: 52.97
Files already downloaded and verified
Files already down

In [0]:
norms = []
for i in range(seeds):
  for j in range(seeds):
    if i != j:
      cross_corr = weights[i] @ weights[j].t()
      norms.append(torch.norm(cross_corr, p='fro'))


In [7]:
print(norms)

[tensor(177.7702, grad_fn=<NormBackward0>), tensor(178.4195, grad_fn=<NormBackward0>), tensor(178.2603, grad_fn=<NormBackward0>), tensor(178.0829, grad_fn=<NormBackward0>), tensor(180.0099, grad_fn=<NormBackward0>), tensor(177.9695, grad_fn=<NormBackward0>), tensor(178.4415, grad_fn=<NormBackward0>), tensor(177.1337, grad_fn=<NormBackward0>), tensor(177.7307, grad_fn=<NormBackward0>), tensor(177.7708, grad_fn=<NormBackward0>), tensor(178.8418, grad_fn=<NormBackward0>), tensor(179.0691, grad_fn=<NormBackward0>), tensor(178.6777, grad_fn=<NormBackward0>), tensor(180.3897, grad_fn=<NormBackward0>), tensor(178.5933, grad_fn=<NormBackward0>), tensor(179.0475, grad_fn=<NormBackward0>), tensor(177.6955, grad_fn=<NormBackward0>), tensor(178.3011, grad_fn=<NormBackward0>), tensor(178.4187, grad_fn=<NormBackward0>), tensor(178.8415, grad_fn=<NormBackward0>), tensor(179.7320, grad_fn=<NormBackward0>), tensor(179.1550, grad_fn=<NormBackward0>), tensor(180.9379, grad_fn=<NormBackward0>), tensor(179

In [8]:
!pip install weightwatcher

  Created wheel for powerlaw: filename=powerlaw-1.4.6-cp36-none-any.whl size=24787 sha256=3119f614f9ace9e57ca5c2d2215a0f7ff8d88b67878cbffb346bb8e402a847c9
  Stored in directory: /root/.cache/pip/wheels/e0/27/02/08d0e2865072bfd8d7c655e290521e3feca3fc22f1ac460601
Successfully built powerlaw


In [9]:
 import weightwatcher as ww

Using TensorFlow backend.


In [10]:
summaries = []

for i in range(9):
  watcher = ww.WeightWatcher(model=models[i])
  results = watcher.analyze(alphas=True, spectralnorms=True, mp_fit=True)
  t = watcher.get_summary()
  summaries.append(t)


2020-04-08 02:51:31,638 INFO 

python      version 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
numpy       version 1.18.2
tensforflow version 2.2.0-rc2
keras       version 2.2.5
2020-04-08 02:51:31,643 INFO Analyzing model
/usr/local/lib/python3.6/dist-packages/powerlaw.py:700: RuntimeWarning: divide by zero encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))
/usr/local/lib/python3.6/dist-packages/powerlaw.py:700: RuntimeWarning: invalid value encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))
2020-04-08 02:51:34,252 INFO ### Printing results ###
2020-04-08 02:51:34,363 INFO Check: min: 0.04609980197267248, max: 0.16807942862578565, avg: 0.10708961529922906
2020-04-08 02:51:34,364 INFO Check compound: min: 0.04609980197267248, max: 0.16807942862578565, avg: 0.10708961529922906
2020-04-08 02:51:34,365 INFO CheckTF: min: False, max: False, avg: 0.0
2020-04-08 02:51:34,367 INFO CheckTF compound: min: 0.0, max: 0.0, avg: 0.0
2020-04-08 02:51:3

In [11]:
for i in range(len(summaries)):
  print("True Labels, iteration", i)
  print('alpha', summaries[i]['alpha'])
  print('spectral norm', summaries[i]['spectralnorm'])
  print('norm', summaries[i]['norm'])

True Labels, iteration 0
alpha 5.420542383139811
spectral norm 29.968962
norm 40.881767
True Labels, iteration 1
alpha 5.67042037735524
spectral norm 30.37325
norm 41.02022
True Labels, iteration 2
alpha 5.508336268209409
spectral norm 30.588753
norm 41.082085
True Labels, iteration 3
alpha 5.318948180348579
spectral norm 30.532377
norm 41.072433
True Labels, iteration 4
alpha 5.307135771190441
spectral norm 29.999058
norm 41.067577
True Labels, iteration 5
alpha 5.133067616126289
spectral norm 31.683002
norm 41.273888
True Labels, iteration 6
alpha 5.661729404774782
spectral norm 30.562023
norm 40.993305
True Labels, iteration 7
alpha 5.440713185179473
spectral norm 30.807962
norm 41.01539
True Labels, iteration 8
alpha 5.288473500517336
spectral norm 29.63554
norm 41.003185
